In [135]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import tensorflow as tf
#Plotly
from plotly.subplots import make_subplots 
import plotly.graph_objects as go

# Import our input dataset
df = pd.read_csv('cleaning3.csv')


# Preprocessing Data For ML learning

In [136]:

df["Job_Title"].value_counts().head(20)

Other                    1547
Data Analyst              442
Senior Data Analyst       131
Junior Data Analyst        47
Business Data Analyst      28
Data Analyst II            20
Lead Data Analyst          19
Data Quality Analyst       18
Name: Job_Title, dtype: int64

In [137]:
df["Job_Title"] = df["Job_Title"].str.replace("Sr. Data Analyst", "Senior Data Analyst")
df["Job_Title"] = df["Job_Title"].str.replace("Sr Data Analyst", "Senior Data Analyst")
df["Job_Title"] = df["Job_Title"].str.replace("Data Analyst Senior", "Senior Data Analyst")
df["Job_Title"] = df["Job_Title"].str.replace('Jr. Data Analyst','Junior Data Analyst')
df["Job_Title"] = df["Job_Title"].str.replace('Jr Data Analyst','Junior Data Analyst')
df["Job_Title"] = df["Job_Title"].str.replace('Data Analyst Junior','Junior Data Analyst')

/Users/YingLuo/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning:

The default value of regex will change from True to False in a future version.

/Users/YingLuo/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning:

The default value of regex will change from True to False in a future version.



In [138]:
#ADD SALARY AVERAGE COLUMN TO DataFrame
df["Salary Average"] = (df["Min_Salary"] + df["Max_Salary"]) / 2

In [139]:
df.head()

,Job_Title,Job_Description,Rating,Company_Name,Location,Size,Founded,Type_of_ownership,Industry,Sector,...,Competitors,Easy_Apply,Department,Min_Salary,Max_Salary,Senior_Job,Salary_Lvl,Salary Average,City,State
0,Data Analyst,Are you eager to roll up your sleeves and harn...,3.2,Vera Institute of Justice,"New York, NY",201 to 500 employees,1961.0,Nonprofit Organization,Social Assistance,Non-Profit,...,NaN,1,Center on Immigration and Justice (CIJ),37,66,False,Low,51.5,New York,NY
1,Other,Overview\n\nProvides analytical and technical ...,3.8,Visiting Nurse Service of New York,"New York, NY",10000+ employees,1893.0,Nonprofit Organization,Health Care Services & Hospitals,Health Care,...,NaN,0,NaN,37,66,False,Low,51.5,New York,NY
2,Senior Data Analyst,We’re looking for a Senior Data Analyst who ha...,3.4,Squarespace,"New York, NY",1001 to 5000 employees,2003.0,Company - Private,Internet,Information Technology,...,GoDaddy,0,Insights & Analytics Team [Customer Operations],37,66,True,Low,51.5,New York,NY
3,Data Analyst,Requisition NumberRR-0001939\nRemote:Yes\nWe c...,4.1,Celerity,"New York, NY",201 to 500 employees,2002.0,Subsidiary or Business Segment,IT Services,Information Technology,...,NaN,0,NaN,37,66,False,Low,51.5,New York,NY
4,Other,ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,3.9,FanDuel,"New York, NY",501 to 1000 employees,2009.0,Company - Private,Sports & Recreation,"Arts, Entertainment & Recreation",...,DraftKings,1,NaN,37,66,False,Low,51.5,New York,NY


In [140]:
from sklearn.decomposition import PCA # For PCA dimensionality reduction
from sklearn.cluster import KMeans    # Method for Cluster Kmeans
from sklearn.preprocessing import QuantileTransformer #Realize Scaler Data
# For visualization from 3D plot
import plotly.express as px
import hvplot.pandas

In [141]:
df2 = df.copy()
df2 = df2.drop(df2[df2['Founded'] == -1].index, axis = 0)
df2 = df2.drop(df2[df2['Rating'] == -1.0].index, axis = 0)
df2['Years Founded'] = (2020 - df2['Founded'])




In [142]:
# Create DataFrame
df3 = pd.DataFrame()
df3 = pd.concat([df2['Rating'], df2['Salary Average'], df2['Years Founded']], axis = 1)
df3.head()

,Rating,Salary Average,Years Founded
0,3.2,51.5,59.0
1,3.8,51.5,127.0
2,3.4,51.5,17.0
3,4.1,51.5,18.0
4,3.9,51.5,11.0


In [143]:
df3=df3.dropna(how="any")

In [144]:
df_scaler = QuantileTransformer().fit_transform(df3)
print(df_scaler)

[[0.16766767 0.16616617 0.8023023 ]
 [0.55005005 0.16616617 0.92495974]
 [0.26576577 0.16616617 0.31381381]
 ...
 [0.07157157 0.81031031 0.87287287]
 [0.10810811 0.81031031 0.66266266]
 [0.26576577 0.81031031 0.95695696]]


In [145]:
# Initialize PCA model
pca = PCA(n_components=2)

In [146]:
# Get two principal components for the  data.
df_components = pca.fit_transform(df_scaler)
df_components.shape,df_scaler.shape

((1573, 2), (1573, 3))

In [147]:
total_var=pca.explained_variance_ratio_.sum()*100
total_var

76.50330020153866

tells us how much of the total variance is explained by each principal component

After this dimensionality reduction, we get a smaller set of dimensions called principal components. These new components are just the two main dimensions of variation that contain most of the information in the original dataset.

In [148]:
#Transform pca data to a DataFrame to fit K-means
df_datajob_pca=pd.DataFrame(
                data=df_components,columns=['Principal 1','Principal 2'])
df_datajob_pca

,Principal 1,Principal 2
0,-0.506952,0.223208
1,-0.327117,0.322643
2,-0.102509,0.265377
3,0.217963,0.396622
4,0.224839,0.357962
...,...,...
1568,-0.472429,-0.419272
1569,-0.095892,-0.411898
1570,-0.482134,-0.424849
1571,-0.312016,-0.408238


In [149]:
 #Find the best value for K
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_datajob_pca)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

elbow curve with the generated principal components showed the K value is 4

In [150]:
# Initialize the K-means model
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(df_datajob_pca)

# Predict clusters
predictions = model.predict(df_datajob_pca)

# Add the predicted class columns to Dataframe
df_datajob_pca["class"] = model.labels_
df_datajob_pca.head()

,Principal 1,Principal 2,class
0,-0.506952,0.223208,1
1,-0.327117,0.322643,1
2,-0.102509,0.265377,1
3,0.217963,0.396622,0
4,0.224839,0.357962,0


In [151]:
predictions

array([1, 1, 1, ..., 3, 3, 3], dtype=int32)

In [152]:
## Graph PCA component with cluster predict
df_datajob_pca.hvplot.scatter(
    x="Principal 1",
    y="Principal 2",
    hover_cols=["class"],
    by="class",
)

:NdOverlay   [class]
   :Scatter   [Principal 1]   (Principal 2,class)

In [153]:

fig3d = px.scatter_3d(df3,x = 'Rating', y = 'Salary Average', z = 'Years Founded', 
                      color = df_datajob_pca['class'],
                      opacity = 0.9, 
                      size_max = 5,
                     title=f'Total Explained Variance: {total_var:.2f}%',)



fig3d.show()

In [ ]:
#  Plot clusters in graph bars to identif classes relt

In [154]:
#create DATAFRAME WITH CLUSTER 
df_scaler=pd.DataFrame(df_scaler)
df_scaler

,0,1,2
0,0.167668,0.166166,0.802302
1,0.550050,0.166166,0.924960
2,0.265766,0.166166,0.313814
3,0.754254,0.166166,0.338839
4,0.590591,0.166166,0.164665
...,...,...,...
1568,0.093093,0.810310,0.880380
1569,0.050551,0.810310,0.291291
1570,0.071572,0.810310,0.872873
1571,0.108108,0.810310,0.662663


In [155]:
merge_df=df_scaler.rename(columns={0:"Rating",1:"Salary Average",2:"Years Founded"})

In [156]:
merge_df['class']=df_datajob_pca['class']
merge_df

,Rating,Salary Average,Years Founded,class
0,0.167668,0.166166,0.802302,1
1,0.550050,0.166166,0.924960,1
2,0.265766,0.166166,0.313814,1
3,0.754254,0.166166,0.338839,0
4,0.590591,0.166166,0.164665,0
...,...,...,...,...
1568,0.093093,0.810310,0.880380,3
1569,0.050551,0.810310,0.291291,3
1570,0.071572,0.810310,0.872873,3
1571,0.108108,0.810310,0.662663,3


In [157]:
#perform groupby 
merge_df=merge_df.groupby(['class']).mean()


In [158]:
merge_df.head()

,Rating,Salary Average,Years Founded
class,,,
0,0.727445,0.261953,0.295705
1,0.333916,0.252238,0.712219
2,0.719264,0.776278,0.250185
3,0.321208,0.736216,0.617517


In [159]:
rename_df=merge_df.rename( index={0: 'class_1',1:'class_2',2:'class_3',3:'class_4'})


In [160]:
rename_df.head()

,Rating,Salary Average,Years Founded
class,,,
class_1,0.727445,0.261953,0.295705
class_2,0.333916,0.252238,0.712219
class_3,0.719264,0.776278,0.250185
class_4,0.321208,0.736216,0.617517


In [161]:
X = list(rename_df.columns)
X

['Rating', 'Salary Average', 'Years Founded']

In [162]:
y = np.array(rename_df)

In [177]:
colors=px.colors.qualitative.G10

In [178]:
# colors = ['crimson','gold', 'darkcyan', 'magenta']

In [179]:
# Plot clusters in graph bars to identify to better
fig_bars = make_subplots(rows = 1, cols = 4, shared_yaxes = True)


# Cluster 1
fig_bars.add_trace(go.Bar(x = X, 
                          y = y[:1,:].tolist()[0],
                          marker = dict(color =colors),
                          name = 'class_1'),
                   row = 1, 
                   col = 1)
# Cluster 2
fig_bars.add_trace(go.Bar(x = X, 
                          y = y[1:2,:].tolist()[0],
                          marker = dict(color = colors),
                          name = 'class_2'),
                   row = 1,
                   col = 2)

# Cluster 3
fig_bars.add_trace(go.Bar(x = X, 
                          y = y[2:3,:].tolist()[0],
                          marker = dict(color = colors), 
                          name = 'class_3'),
                   row = 1,
                   col = 3)

# Cluster 4
fig_bars.add_trace(go.Bar(x =X, 
                          y = y[3:4,:].tolist()[0],
                          marker = dict(color = colors), 
                          name = 'class_4'),
                   row = 1,
                   col = 4)

fig_bars.update_layout(showlegend = False, title = 'Cluster Behavior')

fig_bars.show()